In [ ]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 35.4 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login
login(new_session=False)

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [ ]:
verifier = "google/gemma-3-27b-it"
drafter  = "google/gemma-3-1b-it"

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="fp4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [ ]:
dtype = torch.bfloat16

tokenizer = AutoTokenizer.from_pretrained(verifier, use_fast=True)

draft = AutoModelForCausalLM.from_pretrained(
    drafter,
    device_map="cuda",
    dtype=dtype,
    quantization_config=bnb_config,
)

target = AutoModelForCausalLM.from_pretrained(
    verifier,
    device_map="cuda",
    dtype=dtype,
    quantization_config=bnb_config,
)

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

...

model-00010-of-00012.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00011-of-00012.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00012-of-00012.safetensors:   0%|          | 0.00/462M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/12 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset

# download and load all the pairs
dataset = load_dataset("ai4bharat/IN22-Conv")

README.md:   0%|          | 0.00/7.85k [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/2.01M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/1503 [00:00<?, ? examples/s]

In [ ]:
class SpeculativeDecoder:
    def __init__(self, draft_model, target_model, tokenizer, device):
        self.draft_model = draft_model
        self.target_model = target_model
        self.tokenizer = tokenizer
        self.device = device

    def generate_draft_tokens(self, input_ids, num_tokens):
        draft_tokens = []
        draft_probs = []

        current_ids = input_ids.clone()

        for _ in range(num_tokens):
            with torch.no_grad():
                outputs = self.draft_model(current_ids)
                logits = outputs.logits[0, -1, :]  # Corrected indexing
                probs = torch.softmax(logits, dim=0)

                next_token = torch.multinomial(probs, num_samples=1)
                token_id = next_token.item()

                draft_tokens.append(token_id)
                draft_probs.append(probs[token_id].item())

                current_ids = torch.cat([current_ids, next_token.unsqueeze(0)], dim=1)
        return draft_tokens, draft_probs

    def verify_draft_tokens(self, input_ids: torch.Tensor,
                       draft_tokens: List[int],
                       draft_probs: List[float]) -> List[int]:
        """
        Verify draft tokens using the target model in a single forward pass.

        This is where the magic happens! We process all draft tokens at once
        and get probability distributions at each position.
        """
        # Create sequence with all draft tokens
        draft_sequence = torch.cat([
            input_ids,
            torch.tensor([draft_tokens], device=self.device)
        ], dim=1)

        # Single forward pass through target model
        with torch.no_grad():
            outputs = self.target_model(draft_sequence)
            all_logits = outputs.logits[0]  # Shape: [seq_len, vocab_size]

        # Verify each draft token
        accepted_tokens = []
        seq_len = input_ids.size(1)

        for i in range(len(draft_tokens)):
            # Get target model's probability distribution at this position
            position = seq_len -1 + i # Corrected indexing
            target_probs = torch.softmax(all_logits[position], dim=0)
            target_prob = target_probs[draft_tokens[i]].item()
            draft_prob = draft_probs[i]

            # Acceptance criterion: p_target(token) / p_draft(token)
            acceptance_ratio = min(1.0, target_prob / draft_prob)

            if torch.rand(1).item() < acceptance_ratio:
                # Accept the draft token
                accepted_tokens.append(draft_tokens[i])
            else:
                # Reject and sample from adjusted distribution
                # Adjusted distribution: max(0, p_target - p_draft)
                adjusted_probs = torch.clamp(
                    target_probs - torch.softmax(all_logits[position], dim=0),
                    min=0.0
                )

                if adjusted_probs.sum() > 0:
                    adjusted_probs = adjusted_probs / adjusted_probs.sum()
                    new_token = torch.multinomial(adjusted_probs, num_samples=1).item()
                else:
                    # Fallback: sample from target distribution
                    new_token = torch.multinomial(target_probs, num_samples=1).item()

                accepted_tokens.append(new_token)
                # Stop verifying remaining tokens
                break

        # Bonus token: if all drafts accepted, get one more from target model
        if len(accepted_tokens) == len(draft_tokens):
            position = seq_len - 1 + len(draft_tokens) # Corrected indexing
            bonus_probs = torch.softmax(all_logits[position], dim=0)
            bonus_token = torch.multinomial(bonus_probs, num_samples=1).item()
            accepted_tokens.append(bonus_token)

        return accepted_tokens

    def generate(self, prompt: str, max_new_tokens: int = 50,
                 num_draft_tokens: int = 4, verbose: bool = True) -> str:
        """
        Generate text using speculative decoding.
        """
        input_ids = self.tokenizer.encode(prompt, return_tensors='pt').to(self.device)
        generated_tokens = 0
        iterations = 0
        total_accepted = 0

        while generated_tokens < max_new_tokens:
            iterations += 1

            # Step 1: Draft tokens
            draft_tokens, draft_probs = self.generate_draft_tokens(input_ids, num_draft_tokens)

            # Step 2: Verify drafts
            accepted_tokens = self.verify_draft_tokens(input_ids, draft_tokens, draft_probs)

            # Update statistics
            num_accepted = len(accepted_tokens)
            total_accepted += num_accepted
            generated_tokens += num_accepted

            if verbose:
                draft_text = self.tokenizer.decode(draft_tokens)
                accepted_text = self.tokenizer.decode(accepted_tokens)
                print(f"Iteration {iterations}:")
                print(f"  Drafted: {draft_text!r}")
                print(f"  Accepted: {accepted_text!r} ({num_accepted}/{len(draft_tokens)} tokens)")

            # Add accepted tokens to sequence
            input_ids = torch.cat([
                input_ids,
                torch.tensor([accepted_tokens], device=self.device)
            ], dim=1)

            if generated_tokens >= max_new_tokens:
                break

        result = self.tokenizer.decode(input_ids[0], skip_special_tokens=True)
        return result

    def baseline_generate(self, prompt: str, max_new_tokens: int = 50) -> str:
      """
      Generate text using standard autoregressive decoding (for comparison).

      Args:
          prompt: Input text
          max_new_tokens: Maximum tokens to generate

      Returns:
          Generated text
      """
      input_ids = self.tokenizer.encode(prompt, return_tensors='pt').to(self.device)

      with torch.no_grad():
          output_ids = self.target_model.generate(
              input_ids,
              max_new_tokens=max_new_tokens,
              do_sample=True,
              pad_token_id=self.tokenizer.eos_token_id
          )

      return self.tokenizer.decode(output_ids[0], skip_special_tokens=True)


In [ ]:
m = SpeculativeDecoder(draft, target, tokenizer, device="cuda")
m.baseline_generate("Artificial intelligence is", 64)

'Artificial intelligence is rapidly evolving and is transforming numerous aspects of our lives. While many view it with excitement, others harbor concerns about its potential negative impacts. This article explores the multifaceted relationship between AI and societal well-being, focusing on both the benefits and challenges it presents.\n\n## The Promise of AI for Societal Well-being\n\n'

In [ ]:
import time
prompt = "Artificial intelligence is"
max_tokens = 128

print("\n" + "=" * 80)
print("METHOD 1: SPECULATIVE DECODING")
print("=" * 80)
start_time = time.time()
spec_result = m.generate(prompt, max_new_tokens=max_tokens, num_draft_tokens=3, verbose=False)
spec_time = time.time() - start_time
print(f"Result: {spec_result}")
print(f"Time taken: {spec_time:.2f}s")

# Method 2: Standard Generation
print("\n" + "=" * 80)
print("METHOD 2: STANDARD AUTOREGRESSIVE GENERATION")
print("=" * 80)
start_time = time.time()
baseline_result = m.baseline_generate(prompt, max_new_tokens=max_tokens)
baseline_time = time.time() - start_time
print(f"Result: {baseline_result}")
print(f"Time taken: {baseline_time:.2f}s")

# Summary
print("\n" + "=" * 80)
print("SUMMARY")
print("=" * 80)
print(f"Speculative decoding time: {spec_time:.2f}s")
print(f"Standard generation time: {baseline_time:.2f}s")
print(f"Speedup: {baseline_time / spec_time:.2f}x")
print()


METHOD 1: SPECULATIVE DECODING
Result: Artificial intelligence is rapidly changing the landscape of various industries. This technology is not just unnoticed in the creative realm, particularly in graphic design. AI-powered tools are now assisting designers in streamlining stages, creating unique pieces, and exploring new frontiers. We'll explore the influence of AI on tools, creative process, and the future skills for designers in this post.

**AI Tools for Graphic Design:**

*   **Image Generation:** AI can generate images new with a simple text prompt. Some popular tools include:
    *   **Midjourney:** Known for its highly detailed and artistic imagery.
    *   **DALL-E 2
Time taken: 24.87s

METHOD 2: STANDARD AUTOREGRESSIVE GENERATION
Result: Artificial intelligence is rapidly transforming the world as we know it, with its influence growing exponentially in various sectors. From automating mundane tasks to enabling groundbreaking discoveries, AI is no longer a futuristic concept 